In [1]:
import numpy as np
from joblib import Parallel, delayed

from assignment.util import *
from assignment.TSP import *
from assignment.cooling import *
from assignment.SAMinimizer import *

In [2]:
# logging.basicConfig(level = logging.DEBUG)

In [3]:
cost_eil51, cost_a280, cost_pcb442 = map(first, COST_FUNCTION_PAIRS)
delta_cost_eil51, delta_cost_a280, delta_cost_pcb442 = map(second, COST_FUNCTION_PAIRS)

In [4]:
_, a280_tour = read_tour(OPT_TOUR_FILES[1])
cost_a280(a280_tour)

DEBUG:root:len(self.D) = 280


2586.7696475631606

In [5]:
np.random.seed(0xc0ffee)
cost_function = cost_a280
delta_cost_function = delta_cost_a280
n = delta_cost_function.D.shape[0]
print(n)
T_initial = 100.0
T_final = 0.1

cr = 0.95
cooling_schedule = GeometricCoolingSchedule(cr)
mkov_transition = random_two_opt_transition
state_constructor = Lin2opt

sim_anneal_system = SAMinimizer(
    mkov_transition,
    delta_cost_a280,
    cooling_schedule,
    np.random.uniform,
    state_constructor)

X0 = np.random.permutation(n)
print(X0)

280
[128 103 205  42  64 180  74 110  50  39 225 264 224   2  81 133 100  80
 257 260 176 113 251   8 130 136  17  65 242  25 263   5  93  68  27 239
 155  96 151 279 204  72   6 275  28 132  12  88 109  40 259  34 129 201
  23 241 221 262 145 166 249  94 134 267 209  76 117  45 139  33   9  69
  90 215 219  91  84 148  98 193 268 168 146 220  51   1 178 227 248 254
 208 234 163  92 237  56  24  61 232 228  26 112 274  21  67 230 141 233
 188 186 191  22  87 144  47 190  70 152 258  62  89 194 162 124 149 211
 158 197 138 140  86 159  11  52 183 222 114 250   7 125  57 214 156 218
 238  48 247  18  66 157 160 202   3 121 212  95 143 107 171 192  54 118
 187  19 147 131 199  14 175 119 213  77 246 244   4  15 165 252  97 206
  49 210 240 142  31 278 273 198 115 223  82  59 195 189 216  29 266 200
 122 203 153 243 135 108  73 126  16 120  44 179 150  30 127 261 174 123
  55 184 181  78 106  38 277  58  43 253  46 207 137 272 265  71 245  20
 235 173 177 231 256 111 170 276  99 161 172  1

In [6]:
sim_anneal_system.run(1000, T_initial, X0, cost_a280, StoppingCriterion.MIN_TEMPERATURE, CostProbing.LATEST_GLOBAL_OPTIMUM, T_final)

DEBUG:root:len(self.D) = 280


In [7]:
print(sim_anneal_system.min_cost)

3221.020789518919


In [8]:
NODE_COUNTS = np.array([51, 280, 442])

## Effect of Temperature on Initial Acceptance Rate

In [9]:
def run_batch(sa_system, batch_size, chain_length, T_initial, *args):
    acceptance_rates = np.full(batch_size, np.float64("nan"))
    min_costs = np.full(batch_size, np.float64("nan"))
    node_count = sa_system.delta_cost.D.shape[0]
    for i in range(batch_size):
        X0 = np.random.permutation(node_count)
        sa_system.run(chain_length, T_initial, X0, *args)
        min_costs[i] = sa_system.min_cost
        acceptance_rates[i] = np.float64(sa_system.acceptance_count)/chain_length
        sa_system.reset()
        
    return min_costs, acceptance_rates

In [11]:
# eil51
cost_function, dcost_function = COST_FUNCTION_PAIRS[0]
node_count = cost_function.D.shape[0] 
print(node_count)
cr = 0.95
cooling_schedule = GeometricCoolingSchedule(cr)
mkov_transition = random_two_opt_transition
state_constructor = Lin2opt


c_values = np.linspace(1,10, 10)
T_initial_values = c_values * np.sqrt(node_count)


# for parallelization
N_JOBS = 6

rngs = generate_rngs(len(T_initial_values))


batch_size = 10
min_costs_list = []
chain_length = 1000
chain_count = 100
args = (cost_function, StoppingCriterion.MKOV_CHAIN_COUNT, CostProbing.LATEST_GLOBAL_OPTIMUM, chain_count)


sa_systems = [SAMinimizer(mkov_transition, dcost_function, cooling_schedule, rng.uniform, state_constructor)
                for rng in rngs]


# TODO: parallelize this loop
for i,T0 in enumerate(T_initial_values):
    min_costs, acceptance_rates = run_batch(sa_systems[i], batch_size, chain_length, T0, *args)
    min_cost_lists.append(min_costs)

DEBUG:root:len(self.D) = 51


51


DEBUG:root:len(self.D) = 51
DEBUG:root:len(self.D) = 51
DEBUG:root:len(self.D) = 51


KeyboardInterrupt: 

In [ ]:
SAMinimizer.__init__?